In [1]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
from PMTK.random.preferences_sampler import sample_preferences_from_order, sample_preferences_from_complete_order
from PMTK.utils import *
from PMTK.random.subset_samplers import sample_subsets
from PMTK.utility.additive_utility import AdditiveUtility
from PMTK.preferences import *
from PMTK.utility.utility_fitter import Utility_Fitter
from PMTK.utility.extension_solver import *
from tqdm.notebook import tqdm

In [2]:
def get_all_explicative(x,y):
    s_i = get_all_k_sets(x, len(x))
    s_j = get_all_k_sets(y, len(y))
    theta = []
    for x in s_i:
        if x not in s_j:
            theta.append(x)
    
    for x in s_j:
        if x not in s_i:
            theta.append(x)
    theta = list(set(theta))
    return theta

def equal_lists(L1, L2):
    return all([i in L1 for i in L2]) and all([i in L2 for i in L1])

def exists(e,L):
    for i in L:
        if equal_lists(i,e):
            return True
    return False

def keep_unique(L):
    k = []
    for i in L:
        if not i in k:
            k.append(i)
    return k

def keep_unique_tuples(L):
    k = []
    for i in L:
        if not exists(i,k):
            k.append(i)
    return k


def get_preferences(theta, items, subsets, preferences):
    UF = Utility_Fitter(items, theta)
    UF.set_preferences(preferences).set_model(theta).build_vars().build_preferences_cst()
    p = UF.get_robust_preferences(subsets, verbose=False)
    return p

def get_all_preferences(thetas, items, subsets, preferences):
    D = {}
    for theta in tqdm(thetas):
        p = get_preferences(theta, items, subsets, preferences)
        D[tuple(theta)] = p
    return D

def product(L):
    p = 1
    for i in L:
        p *= i
    return p

def get_all_thetas(preferences):
    explications = []
    for x,y in preferences.preferred:
        c = get_all_explicative(x,y)
        explications.append(c)
    L = [keep_unique(i) for i in itertools.product(*explications)]
    return keep_unique_tuples(L)

def preference_complexity(x,y):
    return 2**len(x) + 2**len(y) - (len(set(x).intersection(set(y))))

def prune_non_compatible_thetas(items, preferences, thetas):
    n_theta = []
    removed_n = 0
    for i in thetas:
        UF = Utility_Fitter(items, i)
        UF.set_preferences(preferences).set_model(i).build_vars().build_preferences_cst().run(UF.get_most_discriminant_utility)
        if UF.get_utility() != None:
            n_theta.append(i)
        else:
            removed_n = removed_n + 1
    return n_theta, removed_n


def run_simulation(items, preferences):
    L = get_all_thetas(preferences)
    theta_prunned, n_removed = prune_non_compatible_thetas(items, preferences, L)
    all_subsets = sorted(get_all_k_sets(items, len(items))+[EMPTY_SET])
    pref_dict = get_all_preferences(theta_prunned, items, all_subsets, preferences)
    return pref_dict, n_removed

def test_compatibility_with_original_prefs(items, preferences, thetas):
    print("Testing the difference between p_1 and preferences")
    for theta in prune_non_compatible_thetas(items, preferences, thetas):
        all_subsets = sorted(get_all_k_sets(items, len(items))+[EMPTY_SET])
        print("========")
        print("Theta:", theta)
        try:
            p_1 = get_preferences(theta, items, all_subsets, preferences)
        except:
            print("Cannot represent the preferences")
            continue
        print(preferences - p_1)
        break
    
def count_contradictions(pref_dict):
    theta_keys = list(pref_dict.keys())
    contradictions = np.zeros((len(pref_dict.keys()), len(pref_dict.keys())))
    produced_preferences = np.zeros(len(theta_keys))
    for theta_1 in pref_dict:
        p_1 = pref_dict[theta_1]
        produced_preferences[theta_keys.index(theta_1)] = len(p_1 - preferences)
        for theta_2 in pref_dict:
            if theta_1 == theta_2:
                continue
            p_2 = pref_dict[theta_2]
            c_1_2 = p_1.contradictions(p_2)
            contradictions[theta_keys.index(theta_1), theta_keys.index(theta_2)] = len(c_1_2)
    return contradictions, produced_preferences

def union_preferences(items, preferences):
    P = Preferences(items)
    for i in preferences:
        P = P + i
    return P

def intersection_preferences(items, preferences):
    P = union_preferences(items, preferences)
    for i in preferences:
        P = P.intersect(i)
    return P

def additivity(theta):
    L = []
    for s in theta:
        L.append(len(s))
    return max(L)
    
def compute_k_min(thetas):
    k = []
    for theta in thetas:
        k.append(additivity(theta))
    return min(k)

def compute_min_size(thetas):
    sizes = []
    for theta in thetas:
        sizes.append(len(theta))
    return min(sizes)

def get_theta_min(thetas):
    k = compute_k_min(thetas)
    theta_min = []
    for theta in thetas:
        if additivity(theta) <= k:
            theta_min.append(theta)
    size_min = compute_min_size(thetas)
    theta_min_2 = []
    for t in theta_min:
        if len(t) <= size_min:
            theta_min_2.append(t)
    return theta_min_2

def inclusion(S1,S2):
    for i in S1:
        if not i in S2:
            return False
    return True

def found_equal(S, thetas):
    for i in range(len(thetas)):
        if inclusion(thetas[i], S) and inclusion(S, thetas[i]):
            return i
    return None

In [3]:
found_equal([0,1], [[0,1], [1,0], [0,2]])

0

In [4]:
def generate_tractable_prefs(n_items, n_preferences, t_min = 10000, t_max = 30000, MAX_ITER = 100):
    cpt = 0
    while(True):
        cpt = cpt + 1
        if(cpt == MAX_ITER):
            print("Reached max iter without finding")
            return None
        n_items = 4
        n_preferences = 5
        items = np.arange(n_items)
        preferences = sample_preferences_from_order(items, n_preferences, indifference_rate=0).sort_by_n_candidates()
        explications = []
        for x,y in preferences.preferred:
            c = get_all_explicative(x,y)
            explications.append(c)
        n = product([len(i) for i in explications])
        if n >= t_min and n <= t_max: 
            return preferences

In [15]:
cpt = 0
datas = []
generated_prefs = []
n_items = 5
n_preferences = 20

for i in range(100):    
    #Generate the instance
    items = np.arange(n_items)
    preferences = generate_tractable_prefs(n_items, n_preferences, t_min = 1000, t_max = 10000)
    
    #Get theta and theta_min
    thetas = get_all_thetas(preferences)
    thetas, removed = prune_non_compatible_thetas(items, preferences, thetas)
    min_thetas = [sorted(i) for i in get_theta_min(thetas)]
    if len(min_thetas) == 1:
        continue
    print("Min thetas:", min_thetas)
    count_ = {}
    s = 0
    for i in range(1000):
        ES = Smallest_Extension_Solver(preferences, [EMPTY_SET])
        ES.extend(verbose = False)
        found_model = tuple(sorted(ES.model[1:]))
        count_[found_model] = count_.get(found_model, 0) + 1
        s += 1 
    
    print("Distribution of produced model: ")
    for x in count_:
        print(f"{x}: {count_[x] / s}")
        
    
    

Min thetas: [[(0,), (1,), (2,)], [(0,), (1,), (3,)]]
Distribution of produced model: 
((0,), (1,), (2,)): 0.505
((0,), (1,), (3,)): 0.495
Min thetas: [[(0,), (1,), (3,)], [(1,), (2,), (3,)], [(0,), (2,), (3,)]]
Distribution of produced model: 
((0,), (2,), (3,)): 0.067
((1,), (2,), (3,)): 0.159
((0,), (1,), (3,)): 0.249
((0,), (1,), (2,), (3,)): 0.525
Min thetas: [[(0,), (1,), (2,)], [(0,), (2,), (3,)], [(1,), (2,), (3,)]]
Distribution of produced model: 
((0,), (1,), (2,)): 0.495
((0,), (2,), (3,)): 0.259
((1,), (2,), (3,)): 0.246
Min thetas: [[(0,), (1,), (2,)], [(0,), (1,), (3,)]]
Distribution of produced model: 
((0,), (1,), (2,), (3,)): 0.639
((0,), (1,), (2,)): 0.178
((0,), (1,), (3,)): 0.183
Min thetas: [[(0,), (2,), (3,)], [(0,), (1,), (3,)], [(1,), (2,), (3,)]]
Distribution of produced model: 
((0,), (2,), (3,)): 0.23
((1,), (2,), (3,)): 0.253
((0,), (1,), (3,)): 0.517
Min thetas: [[(0,), (2,), (3,)], [(0,), (1,), (3,)], [(1,), (2,), (3,)]]
Distribution of produced model: 
((0